# anomly-detection - a fast anomaly detection library for hierarchical timeseries

## Introduction

`anomaly-detection` builds on Facebook's Prophet algorithm to identify unusual outliers and trends within hierarchical time-series data. In only a few lines of code, you can flag outliers and changepoints in your hierarchical time-series data.


The example provided in this notebook were created using filtered data from the [M5 competition](https://www.kaggle.com/c/m5-forecasting-accuracy). It's intended as a reference for how this library is intended to be used, and will be updated frequently as we add more features.

## Load Data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import numpy as np
import pandas as pd
from anomaly_detection import AnomalyDetector

In [3]:
data = pd.read_csv(
        "https://raw.githubusercontent.com/facebookincubator/prophet/master/examples/example_retail_sales.csv"
    )

### Sample Data

In [4]:
data

,ds,y
0,1992-01-01,146376
1,1992-02-01,147079
2,1992-03-01,159336
3,1992-04-01,163669
4,1992-05-01,170068
...,...,...
288,2016-01-01,400928
289,2016-02-01,413554
290,2016-03-01,460093
291,2016-04-01,450935


## Initialization

In [5]:
detector = AnomalyDetector(data=data, )

## Detect Anomalies & Plot Results

Behind the scenes, .detect_anomalies() fits Prophet's modeling pipeline to create trend curves and 95% confidence intervals that can be used to detect unusual behavior.

In [6]:
detector.detect_anomalies()
detector.plot_anomalies()

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


alt.LayerChart(...)

To control the flexibility of your model, you can pass arguments directly to Prophet's `.fit()` method:

In [7]:
?detector.fit

Signature: detector.fit(interval_width=0.95, changepoint_range=1, *args, **kwargs)
Docstring:
Fit Prophet to the data.

Parameters (passed as *args or **kwargs to Prophet)
----------
interval_width: float, default .95
    Float, width of the uncertainty intervals provided
    for the forecast. If mcmc_samples=0, this will be only the uncertainty
    in the trend using the MAP estimate of the extrapolated generative
    model. If mcmc.samples>0, this will be integrated over all model
    parameters, which will include uncertainty in seasonality. In this library,
    we override FB's default from .8 to .95 to provide more stringer
    anomaly detection.
growth: str, default "linear"
    String 'linear' or 'logistic' to specify a linear or logistic trend.
changepoints: list, default None 
    List of dates at which to include potential changepoints. If
    not specified, potential changepoints are selected automatically.
n_changepoints: int, default 25
    Number of potential changepoints

In [8]:
detector = AnomalyDetector(data=data) # Initialize a new object, otherwise you have to run .fit() and .predict() on your second run
detector.detect_anomalies(interval_width=.8, seasonality_mode="multiplicative")
detector.plot_anomalies()

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


alt.LayerChart(...)

## Append Results

When you're finished, you can get back your original dataframe by running `.get_results`. 

In [9]:
final_results = detector.get_results()
final_results

,ds,y,anomaly_score,changepoint_flag
0,1992-01-01,146376,0.000000,0
1,1992-02-01,147079,0.000000,0
2,1992-03-01,159336,-0.002718,0
3,1992-04-01,163669,0.000000,0
4,1992-05-01,170068,0.000000,0
...,...,...,...,...
288,2016-01-01,400928,0.000000,0
289,2016-02-01,413554,0.000000,0
290,2016-03-01,460093,0.000000,0
291,2016-04-01,450935,0.000000,0
